In [58]:
import math
import pymysql
import pandas as pd
import pprint


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# import wikipedia
# import spacy
from textblob import TextBlob
from IPython.display import Image
from collections import defaultdict

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from scipy import sparse
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
restaurants_df = pd.read_csv('Sample_Restaurants.csv', index_col='Users')

In [60]:
restaurants_df.fillna(value=0, inplace=True)

In [61]:
restaurants_df.head()

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0


In [62]:
restaurants_df.loc[restaurants_df.index=='Ajinth']

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0


In [63]:
restaurants_df[(restaurants_df > 0).any(axis=1)]

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0
Chris,3.0,5.0,0.0,0.0,0.0,4.0,0.0


# Collaborative Item to Item Filtering

In [64]:
'''List of All Restaurants from the Dataframe'''
all_restaurants = list(restaurants_df.columns)
all_restaurants

['Little_Donkey',
 'Caf__Nero',
 'India_Pavilion',
 'Desi_Dhaba',
 'Thai_Sensation',
 'Dumpling_House',
 'Caf__Luna']

In [65]:
user = 'Tess'

In [66]:
'''Function to return the list of restaurants rated by the user'''
def user_rated_list(user):
    user_rated=[]
    for key, a_res in enumerate(list(restaurants_df.loc[user, :] ==0)):
        if a_res == False:
            user_rated.append((restaurants_df.loc[user, :] ==0).index[key])
        else: 
            continue
    return user_rated   

In [67]:
user_rated = user_rated_list(user)
user_rated

['Little_Donkey', 'Caf__Nero', 'India_Pavilion', 'Thai_Sensation']

In [68]:
'''Function to return the list of restaurants yet to be rated by the user'''
def user_not_rated_list(user):
    user_not_rated=[]
    for key, a_res in enumerate(list(restaurants_df.loc[user, :] ==0)):
        if a_res == True:
            user_not_rated.append((restaurants_df.loc[user, :] ==0).index[key])
        else: 
            continue
    return (user_not_rated) 

In [69]:
user_not_rated = user_not_rated_list(user)
user_not_rated

['Desi_Dhaba', 'Dumpling_House', 'Caf__Luna']

In [70]:
'''Custom Function to calculate the Co-Similarity'''
def cosim(v1, v2):
    dot_product = sum(n1 * n2 for n1, n2 in zip(v1, v2) )
    magnitude1 = math.sqrt(sum(n ** 2 for n in v1))
    magnitude2 = math.sqrt(sum(n ** 2 for n in v2))
    return dot_product / (magnitude1 * magnitude2)

In [78]:
a = restaurants_df['Little_Donkey']
type(a)

pandas.core.series.Series

In [74]:
b = restaurants_df['Caf__Nero']
b

Users
Ajinth      0.0
Tess        3.0
Jessica     3.0
Mark        2.0
Joe         0.0
Chris       5.0
Name: Caf__Nero, dtype: float64

In [77]:
cosim(a,b)

0.6063493831678936

In [14]:
'''This function calcultes the co-sine similarity between the restaurants rsted by the user and all the restaurants in the list'''
def cooccurence_matrix_creator():
    '''Calling the user rated function'''
    user_rated = user_rated_list(user)
    ur_rest_dict = {}
    for ur in user_rated: 
        ur_cosim=[]
        for ar in list(restaurants_df.columns): 
            similarity = cosim(restaurants_df[ur],\
                                            restaurants_df[ar])
            ur_cosim.append(similarity)
        ur_rest_dict[ur] = ur_cosim
    co_occurence_matrix = pd.DataFrame.from_dict(ur_rest_dict, orient='index')
    co_occurence_matrix.columns=list(restaurants_df.columns)
    return co_occurence_matrix

In [15]:
co_occurence_matrix = cooccurence_matrix_creator()
co_occurence_matrix.head()

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Little_Donkey,1.000000,0.606349,0.811778,0.00000,0.862105,0.309839,0.651273
Caf__Nero,0.606349,1.000000,0.170910,0.29173,0.272259,0.782794,0.174034
India_Pavilion,0.811778,0.170910,1.000000,0.00000,0.904497,0.000000,0.706049
Thai_Sensation,0.862105,0.272259,0.904497,0.00000,1.000000,0.000000,0.809807


# Review of some of the Variables

In [16]:
user_rated

['Little_Donkey', 'Caf__Nero', 'India_Pavilion', 'Thai_Sensation']

In [17]:
user_not_rated

['Desi_Dhaba', 'Dumpling_House', 'Caf__Luna']

In [18]:
co_occurence_matrix

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Little_Donkey,1.000000,0.606349,0.811778,0.00000,0.862105,0.309839,0.651273
Caf__Nero,0.606349,1.000000,0.170910,0.29173,0.272259,0.782794,0.174034
India_Pavilion,0.811778,0.170910,1.000000,0.00000,0.904497,0.000000,0.706049
Thai_Sensation,0.862105,0.272259,0.904497,0.00000,1.000000,0.000000,0.809807


In [19]:
def rest_to_rate(df):
    restaurants_to_rate = df[user_not_rated_list(user)]
    return restaurants_to_rate

In [20]:
restaurants_to_rate = rest_to_rate(co_occurence_matrix)
restaurants_to_rate

,Desi_Dhaba,Dumpling_House,Caf__Luna
Little_Donkey,0.00000,0.309839,0.651273
Caf__Nero,0.29173,0.782794,0.174034
India_Pavilion,0.00000,0.000000,0.706049
Thai_Sensation,0.00000,0.000000,0.809807


In [21]:
'''Generating the prediction'''

def rating_predictor_colab_ii():
    predicted_rating_dict = {}
    for not_rated in restaurants_to_rate.columns: 
        column_df = pd.DataFrame(restaurants_to_rate[not_rated].sort_values(ascending=False))
        predicted_rating = []
        top_sim_list = []
        for top_n in range(2):   

            top_sim = column_df[not_rated][top_n]
            top_sim_list.append(top_sim)

            given_rr = column_df[not_rated].index[top_n]
            given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
            given_rank = given_rank_df.loc[given_rr,:]

            predicted_rating_item = float(given_rank*top_sim)
            predicted_rating.append(predicted_rating_item)
        total_sim_weight = sum(top_sim_list)
        adjusted_predicted_rating = (sum(predicted_rating))/total_sim_weight
        predicted_rating_dict[not_rated] = adjusted_predicted_rating
        
    '''Creating the Dataframe in this step'''
        
    predicted_df = pd.DataFrame.from_dict(predicted_rating_dict, orient='index')
    predicted_df.columns=[user]
    return predicted_df
#     return (predicted_rating_dict)


In [22]:
predicted_df = rating_predictor_colab_ii()
predicted_df.head()

,Tess
Desi_Dhaba,3.000000
Dumpling_House,3.000000
Caf__Luna,3.534224


In [23]:
# '''Generating the prediction - All in One Function'''

# def rating_predictor_colab_ii_all():
    
#     predicted_rating_dict = {}
    
# #     user_rated = user_rated_list(user)
# #     user_not_rated = user_not_rated_list(user)
# #     
#     co_occurence_matrix = cooccurence_matrix_creator()
#     restaurants_to_rate = rest_to_rate(co_occurence_matrix)
    
#     for not_rated in restaurants_to_rate.columns: 
#         column_df = pd.DataFrame(restaurants_to_rate[not_rated].sort_values(ascending=False))
#         predicted_rating = []
#         top_sim_list = []
#         for top_n in range(2):   

#             top_sim = column_df[not_rated][top_n]
#             top_sim_list.append(top_sim)

#             given_rr = column_df[not_rated].index[top_n]
#             given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
#             given_rank = given_rank_df.loc[given_rr,:]

#             predicted_rating_item = float(given_rank*top_sim)
#             predicted_rating.append(predicted_rating_item)
#         total_sim_weight = sum(top_sim_list)
#         adjusted_predicted_rating = (sum(predicted_rating))/total_sim_weight
#         predicted_rating_dict[not_rated] = adjusted_predicted_rating
        
#     '''Creating the Dataframe in this step'''
        
#     predicted_df = pd.DataFrame.from_dict(predicted_rating_dict, orient='index')
#     predicted_df.columns=[user]
#     return predicted_df
# #     return (predicted_rating_dict)

In [24]:
co_occurence_matrix

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Little_Donkey,1.000000,0.606349,0.811778,0.00000,0.862105,0.309839,0.651273
Caf__Nero,0.606349,1.000000,0.170910,0.29173,0.272259,0.782794,0.174034
India_Pavilion,0.811778,0.170910,1.000000,0.00000,0.904497,0.000000,0.706049
Thai_Sensation,0.862105,0.272259,0.904497,0.00000,1.000000,0.000000,0.809807


In [25]:
'''Bugs Fixed in the main dataset run

1. Added condition to handle users who have rated only one restaurant - Was getting index out of bound error in the for loop
2. Adding logic to handle folks who have rated multiple restaurants but the coocurence is non-zero for only on restaurant

''' 

'''Collaborative Item-Item filtering function to make predictions'''

def rating_predictor_colab_ii_all():
    
    predicted_rating_dict = {}
    
#     user_rated = user_rated_list(user)
#     user_not_rated = user_not_rated_list(user)
     
    co_occurence_matrix = cooccurence_matrix_creator()
    restaurants_to_rate = rest_to_rate(co_occurence_matrix)
    
    for not_rated in restaurants_to_rate.columns: 
        column_df = pd.DataFrame(restaurants_to_rate[not_rated].sort_values(ascending=False))
        predicted_rating = []
        top_sim_list = []
        if column_df.shape[0] == 1: 
            top_sim = column_df[not_rated]            
            top_sim_list.append(top_sim)

            given_rr = column_df[not_rated].index
            
            given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
            given_rank = float((given_rank_df.loc[given_rr,:]).get_values())

            adjusted_predicted_rating = float(given_rank*top_sim)
            predicted_rating_dict[not_rated] = adjusted_predicted_rating
             
        else: 
            for top_n in range(co_occurence_matrix.shape[0]):   

                top_sim = column_df[not_rated][top_n]
                
                '''Added for bug fix'''
                if top_sim == 0: 
                    continue 
                else:
                    top_sim_list.append(top_sim)

                    given_rr = column_df[not_rated].index[top_n]

                    given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
                    given_rank = given_rank_df.loc[given_rr,:]

                    predicted_rating_item = float(given_rank*top_sim)
                    predicted_rating.append(predicted_rating_item)
            
            if len(top_sim_list) == 1: 
                predicted_rating_dict[not_rated] = predicted_rating_item
            
            else: 

                total_sim_weight = sum(top_sim_list)
#                 print ('Total_sim_weight:', total_sim_weight)
                adjusted_predicted_rating = (sum(predicted_rating))/total_sim_weight
                predicted_rating_dict[not_rated] = adjusted_predicted_rating
        
    '''Creating the Dataframe in this step'''
        
    predicted_df = pd.DataFrame.from_dict(predicted_rating_dict, orient='index')
    predicted_df.fillna(0, inplace=True)
    predicted_df.columns=[user]
    predicted_df.sort_values([user], ascending=False, inplace=True)
    return predicted_df


In [26]:
predicted_df = rating_predictor_colab_ii_all()
predicted_df.head()

,Tess
Caf__Luna,3.345899
Dumpling_House,3.000000
Desi_Dhaba,0.875190


# Collaborative User to User Filtering

In [27]:
restaurants_df

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0
Chris,3.0,5.0,0.0,0.0,0.0,4.0,0.0


In [28]:
# user_not_rated

In [29]:
'''This function calcultes the co-sine similarity between different users for the user-user collaborative filtering model'''

def user_cooccurence_matrix_creator():
    user_dict = {}
    user_columns = []
    user_cosim=[]
    
    for ur in restaurants_df.iterrows(): 
        if ur[0] == user: 
            continue 
        else: 
            user_similarity = cosim(restaurants_df.loc[ur[0],:].values, restaurants_df.loc[user,:].values)
            user_cosim.append(user_similarity)
            user_columns.append(ur[0])
    user_dict[user] = user_cosim
    
    user_df = pd.DataFrame.from_dict(user_dict, orient='index')
    user_df.columns = user_columns
    return user_df
    


In [30]:
ur_coocurence_df = user_cooccurence_matrix_creator()
ur_coocurence_df

,Ajinth,Jessica,Mark,Joe,Chris
Tess,0.809345,0.807427,0.172917,0.708676,0.517597


In [31]:
'''Sorting the dataframe here in descending order'''
ur_coocurence_df.sort_index(ascending=False, axis=0, inplace=True)
ur_coocurence_df

,Ajinth,Jessica,Mark,Joe,Chris
Tess,0.809345,0.807427,0.172917,0.708676,0.517597


In [32]:
ur_coocurence_df_t= ur_coocurence_df.T
ur_coocurence_df_t.columns=['Similarity']
ur_coocurence_df_t

,Similarity
Ajinth,0.809345
Jessica,0.807427
Mark,0.172917
Joe,0.708676
Chris,0.517597


In [33]:
ur_restaurants_to_rate = rest_to_rate(restaurants_df)
ur_restaurants_to_rate

,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Ajinth,0.0,0.0,2.0
Tess,0.0,0.0,0.0
Jessica,0.0,0.0,1.0
Mark,2.0,2.0,4.0
Joe,0.0,0.0,8.0
Chris,0.0,4.0,0.0


In [34]:
ur_restaurants_to_rate.drop(user, axis = 0, inplace=True )
ur_restaurants_to_rate

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Ajinth,0.0,0.0,2.0
Jessica,0.0,0.0,1.0
Mark,2.0,2.0,4.0
Joe,0.0,0.0,8.0
Chris,0.0,4.0,0.0


In [35]:
joint_matrix = ur_restaurants_to_rate.join(ur_coocurence_df_t, how='inner')
# joint_df = joint_matrix.iloc[:2,:]
joint_df = joint_matrix.iloc[:3,:]
joint_df

,Desi_Dhaba,Dumpling_House,Caf__Luna,Similarity
Users,,,,
Ajinth,0.0,0.0,2.0,0.809345
Jessica,0.0,0.0,1.0,0.807427
Mark,2.0,2.0,4.0,0.172917


In [36]:
'''Working on some more updates to the cell above'''

rating_predictor_df = defaultdict(list)
for k,v in enumerate(joint_df.iterrows()): 
    for i in range(len(v[1].index)):
        rating = (v[1][i]) 
        restaurant = (v[1].index[i])  
        rating_predictor_df[restaurant].append(rating)
        
pprint.pprint (rating_predictor_df)

defaultdict(<class 'list'>,
            {'Caf__Luna': [2.0, 1.0, 4.0],
             'Desi_Dhaba': [0.0, 0.0, 2.0],
             'Dumpling_House': [0.0, 0.0, 2.0],
             'Similarity': [0.80934464827409736,
                            0.80742676726507157,
                            0.17291712531127049]})


In [37]:
rating_dict = defaultdict(int)

for k,v in rating_predictor_df.items():
    print ('Key, Value', k,v)
    print (rating_predictor_df['Similarity'])
    new_rating = sum(np.array(v)*np.array(rating_predictor_df['Similarity']))/sum(np.array(rating_predictor_df['Similarity']))
    rating_dict[k] = new_rating
    
print (rating_dict)

Key, Value Desi_Dhaba [0.0, 0.0, 2.0]
[0.80934464827409736, 0.80742676726507157, 0.17291712531127049]
Key, Value Dumpling_House [0.0, 0.0, 2.0]
[0.80934464827409736, 0.80742676726507157, 0.17291712531127049]
Key, Value Caf__Luna [2.0, 1.0, 4.0]
[0.80934464827409736, 0.80742676726507157, 0.17291712531127049]
Key, Value Similarity [0.80934464827409736, 0.80742676726507157, 0.17291712531127049]
[0.80934464827409736, 0.80742676726507157, 0.17291712531127049]
defaultdict(<class 'int'>, {'Desi_Dhaba': 0.19323711513412525, 'Dumpling_House': 0.19323711513412525, 'Caf__Luna': 1.7420822081013119, 'Similarity': 0.74698867758112342})


In [38]:
'''Making Predictions for the User - User Collaborative Filtering'''
rating_dict = defaultdict(int)

for k,v in rating_predictor_df.items():
    new_rating = sum(np.array(v)*np.array(rating_predictor_df['Similarity']))/sum(np.array(rating_predictor_df['Similarity']))
    rating_dict[k] = new_rating

pprint.pprint (rating_dict)

rating_df = pd.DataFrame.from_dict(rating_dict, orient='index')
rating_df.columns=[user]
rating_df.sort_values(user, ascending=False, inplace=True)
rating_df.head()  

defaultdict(<class 'int'>,
            {'Caf__Luna': 1.7420822081013119,
             'Desi_Dhaba': 0.19323711513412525,
             'Dumpling_House': 0.19323711513412525,
             'Similarity': 0.74698867758112342})


,Tess
Caf__Luna,1.742082
Similarity,0.746989
Desi_Dhaba,0.193237
Dumpling_House,0.193237


# Helper Functions for User-User Collaborative Filtering Model

In [39]:
'''Co-occurence Matrix Creator for User-User Filtering'''

def user_cooccurence_matrix_creator(df):
    user_dict = {}
    user_columns = []
    user_cosim=[]
    
    for ur in df.iterrows(): 
        if ur[0] == user: 
            continue 
        else: 
            user_similarity = cosim(df.loc[ur[0],:].values, df.loc[user,:].values)
            user_cosim.append(user_similarity)
            user_columns.append(ur[0])
    user_dict[user] = user_cosim
    
    user_df = pd.DataFrame.from_dict(user_dict, orient='index')
    user_df.columns = user_columns
    return user_df

In [40]:
'''Calling the User Co-Occurence Matrix Function'''
user_user_coc = user_cooccurence_matrix_creator(restaurants_df)
user_user_coc

,Ajinth,Jessica,Mark,Joe,Chris
Tess,0.809345,0.807427,0.172917,0.708676,0.517597


In [41]:
def dataframe_tranposer(df):
    ur_coocurence_df_t = df.T
    ur_coocurence_df_t.columns=['Similarity']
    return ur_coocurence_df_t
    

In [42]:
'''Calling the Transposer Function'''
user_user_cocmatrix_t = dataframe_tranposer(user_user_coc)
user_user_cocmatrix_t

,Similarity
Ajinth,0.809345
Jessica,0.807427
Mark,0.172917
Joe,0.708676
Chris,0.517597


In [43]:
ur_restaurants_to_rate = rest_to_rate(restaurants_df)
ur_restaurants_to_rate

,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Ajinth,0.0,0.0,2.0
Tess,0.0,0.0,0.0
Jessica,0.0,0.0,1.0
Mark,2.0,2.0,4.0
Joe,0.0,0.0,8.0
Chris,0.0,4.0,0.0


In [44]:
'''Dropping the Self User '''

def drop_self_user(df): 
    df.drop(user, axis = 0, inplace=True )
    return df

In [45]:
drop_self_user(ur_restaurants_to_rate)

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Ajinth,0.0,0.0,2.0
Jessica,0.0,0.0,1.0
Mark,2.0,2.0,4.0
Joe,0.0,0.0,8.0
Chris,0.0,4.0,0.0


In [46]:
ur_restaurants_to_rate

,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Ajinth,0.0,0.0,2.0
Jessica,0.0,0.0,1.0
Mark,2.0,2.0,4.0
Joe,0.0,0.0,8.0
Chris,0.0,4.0,0.0


In [47]:
print (ur_restaurants_to_rate.index)
print (user_user_cocmatrix_t.index)

Index(['Ajinth', 'Jessica ', 'Mark', 'Joe', 'Chris'], dtype='object', name='Users')
Index(['Ajinth', 'Jessica ', 'Mark', 'Joe', 'Chris'], dtype='object')


In [48]:
'''Creating the joint matrix between the user similarity and the user rating'''

def joint_matrix_creator(restaurant_to_rate_df, cooccurence_df):
    joint_matrix = restaurant_to_rate_df.join(cooccurence_df, how='inner')
    joint_matrix.sort_values('Similarity', ascending=False, inplace=True)
    return joint_matrix

In [49]:
joint_df = joint_matrix_creator(ur_restaurants_to_rate,user_user_cocmatrix_t )
joint_df

,Desi_Dhaba,Dumpling_House,Caf__Luna,Similarity
Users,,,,
Ajinth,0.0,0.0,2.0,0.809345
Jessica,0.0,0.0,1.0,0.807427
Joe,0.0,0.0,8.0,0.708676
Chris,0.0,4.0,0.0,0.517597
Mark,2.0,2.0,4.0,0.172917


In [50]:
'''Function to create an interim dictionary with the user similarities and the restaurants to be rated for the user'''

def rating_predictor_dict_creator(df): 
    
    rating_predictor_dict = defaultdict(list)
    
    for k,v in enumerate(df.iterrows()): 
        for i in range(len(v[1].index)):
            rating = (v[1][i]) 
            restaurant = (v[1].index[i])  
            rating_predictor_dict[restaurant].append(rating)
    return rating_predictor_dict
    

In [51]:
rating_predictor_df = rating_predictor_dict_creator(joint_df)
pprint.pprint (rating_predictor_dict)

NameError: name 'rating_predictor_dict' is not defined

In [52]:
'''Making Predictions for the User - User Collaborative Filtering'''
rating_dict = defaultdict(int)

for k,v in rating_predictor_df.items():
    new_rating = sum(np.array(v)*np.array(rating_predictor_df['Similarity']))/sum(np.array(rating_predictor_df['Similarity']))
    rating_dict[k] = new_rating

pprint.pprint (rating_dict)

rating_df = pd.DataFrame.from_dict(rating_dict, orient='index')
rating_df.columns=[user]
rating_df.sort_values(user, ascending=False, inplace=True)
rating_df.head() 

defaultdict(<class 'int'>,
            {'Caf__Luna': 2.9135615100560242,
             'Desi_Dhaba': 0.11466799551437509,
             'Dumpling_House': 0.80114535942305143,
             'Similarity': 0.69861805917288566})


,Tess
Caf__Luna,2.913562
Dumpling_House,0.801145
Similarity,0.698618
Desi_Dhaba,0.114668


In [53]:
def rating_predictor_colab_uu_all(dictionary):
    
    rating_dict = defaultdict(int)

    for k,v in dictionary.items():
        new_rating = sum(np.array(v)*np.array(dictionary['Similarity']))/sum(np.array(dictionary['Similarity']))
        rating_dict[k] = new_rating

    pprint.pprint (rating_dict)

    prediction_df = pd.DataFrame.from_dict(rating_dict, orient='index')
    prediction_df.columns=[user]
    prediction_df.sort_values(user, ascending=False, inplace=True)
    return prediction_df
     

# Collaborative User-User filtering (Function)

In [54]:
user='Tess'

# Helper functions for the collaborative user-user filtering model

In [55]:
'''Co-occurence Matrix Creator for User-User Filtering'''

def user_cooccurence_matrix_creator(df):
    user_dict = {}
    user_columns = []
    user_cosim=[]
    
    for ur in df.iterrows(): 
        if ur[0] == user: 
            continue 
        else: 
            user_similarity = cosim(df.loc[ur[0],:].values, df.loc[user,:].values)
            user_cosim.append(user_similarity)
            user_columns.append(ur[0])
    user_dict[user] = user_cosim
    
    user_df = pd.DataFrame.from_dict(user_dict, orient='index')
    user_df.columns = user_columns
    return user_df

'''Function to tranpose the Dataframe'''

def dataframe_tranposer(df):
    ur_coocurence_df_t = df.T
    ur_coocurence_df_t.columns=['Similarity']
    return ur_coocurence_df_t

'''Function to drop the User'''

def drop_self_user(df): 
    df.drop(user, axis = 0, inplace=True )
    return df

'''Creating the joint matrix'''

def joint_matrix_creator(restaurant_to_rate_df, cooccurence_df):
    joint_matrix = restaurant_to_rate_df.join(cooccurence_df, how='inner')
    joint_matrix.sort_values('Similarity', ascending=False, inplace=True)
    return joint_matrix

'''Function to create an interim dictionary with the user similarities and the restaurants to be rated for the user'''

def rating_predictor_dict_creator(df): 
    
    rating_predictor_dict = defaultdict(list)
    
    for k,v in enumerate(df.iterrows()): 
        for i in range(len(v[1].index)):
            rating = (v[1][i]) 
            restaurant = (v[1].index[i])  
            rating_predictor_dict[restaurant].append(rating)
    return rating_predictor_dict
    

In [56]:
# Making the predictions 

In [57]:
'''Mother of all Predictors'''

def rating_predictor_colab_uu_all():
    
    user_user_coc = user_cooccurence_matrix_creator(restaurants_df)
    user_user_cocmatrix_t = dataframe_tranposer(user_user_coc)
    ur_restaurants_to_rate_all = rest_to_rate(restaurants_df)
    ur_restaurants_to_rate = drop_self_user(ur_restaurants_to_rate_all)
    joint_df = joint_matrix_creator(ur_restaurants_to_rate,user_user_cocmatrix_t)
    rating_predictor_dict = rating_predictor_dict_creator(joint_df)
    
    rating_dict = defaultdict(int)

    for k,v in rating_predictor_dict.items():
        
        if k =='Similarity': 
            continue
        else: 
            new_rating = sum(np.array(v)*np.array(rating_predictor_dict['Similarity']))/sum(np.array(rating_predictor_dict['Similarity']))
            rating_dict[k] = new_rating

    prediction_df = pd.DataFrame.from_dict(rating_dict, orient='index')
    prediction_df.columns=[user]
    prediction_df.sort_values(user, ascending=False, inplace=True)
    return prediction_df

In [66]:
prediction = rating_predictor_colab_uu_all()
prediction.head()

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Tess
Caf__Luna,2.913562
Dumpling_House,0.801145
Desi_Dhaba,0.114668


In [28]:
user_rated_list(user)

['Little_Donkey', 'Caf__Nero', 'India_Pavilion', 'Thai_Sensation']

In [29]:
user_not_rated_list(user)

['Desi_Dhaba', 'Dumpling_House ', 'Caf__Luna']

In [32]:
restaurants_df[restaurants_df.index=='Tess']

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
